In [ ]:
!!pip install litellm


# Set Groq API credentials and model directly
import litellm

#api_key = "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"
api_key = "API KEY"
model = "gorq/meta-llama/llama-4-scout-17b-16e-instruct"
base_url = "https://api.groq.com/openai/v1"


In [ ]:
import json
import time
import traceback
import inspect
from litellm import completion
from dataclasses import dataclass, field
from typing import get_type_hints, List, Callable, Dict, Any, Optional

tools = {}
tools_by_tag = {}


def to_openai_tools(tools_metadata: List[dict]):
    openai_tools = [
        {
            "type": "function",
            "function": {
                "name": t['tool_name'],
                # Include up to 1024 characters of the description
                "description": t.get('description',"")[:1024],
                "parameters": t.get('parameters',{}),
            },
        } for t in tools_metadata
    ]
    return openai_tools

def get_tool_metadata(func, tool_name=None, description=None, parameters_override=None, terminal=False, tags=None):
    """
    Extracts metadata for a function to use in tool registration.

    Parameters:
        func (function): The function to extract metadata from.
        tool_name (str, optional): The name of the tool. Defaults to the function name.
        description (str, optional): Description of the tool. Defaults to the function's docstring.
        parameters_override (dict, optional): Override for the argument schema. Defaults to dynamically inferred schema.
        terminal (bool, optional): Whether the tool is terminal. Defaults to False.
        tags (List[str], optional): List of tags to associate with the tool.

    Returns:
        dict: A dictionary containing metadata about the tool, including description, args schema, and the function.
    """
    # Default tool_name to the function name if not provided
    tool_name = tool_name or func.__name__

    # Default description to the function's docstring if not provided
    description = description or (func.__doc__.strip() if func.__doc__ else "No description provided.")

    # Discover the function's signature and type hints if no args_override is provided
    if parameters_override is None:
        signature = inspect.signature(func)
        type_hints = get_type_hints(func)

        # Build the arguments schema dynamically
        args_schema = {
            "type": "object",
            "properties": {},
            "required": []
        }
        for param_name, param in signature.parameters.items():

            if param_name in ["action_context", "action_agent"]:
                continue  # Skip these parameters

            def get_json_type(param_type):
                if param_type == str:
                    return "string"
                elif param_type == int:
                    return "integer"
                elif param_type == float:
                    return "number"
                elif param_type == bool:
                    return "boolean"
                elif param_type == list:
                    return "array"
                elif param_type == dict:
                    return "object"
                else:
                    return "string"

            # Add parameter details
            param_type = type_hints.get(param_name, str)  # Default to string if type is not annotated
            param_schema = {"type": get_json_type(param_type)}  # Convert Python types to JSON schema types

            args_schema["properties"][param_name] = param_schema

            # Add to required if not defaulted
            if param.default == inspect.Parameter.empty:
                args_schema["required"].append(param_name)
    else:
        args_schema = parameters_override

    # Return the metadata as a dictionary
    return {
        "tool_name": tool_name,
        "description": description,
        "parameters": args_schema,
        "function": func,
        "terminal": terminal,
        "tags": tags or []
    }


def register_tool(tool_name=None, description=None, parameters_override=None, terminal=False, tags=None):
    """
    A decorator to dynamically register a function in the tools dictionary with its parameters, schema, and docstring.

    Parameters:
        tool_name (str, optional): The name of the tool to register. Defaults to the function name.
        description (str, optional): Override for the tool's description. Defaults to the function's docstring.
        parameters_override (dict, optional): Override for the argument schema. Defaults to dynamically inferred schema.
        terminal (bool, optional): Whether the tool is terminal. Defaults to False.
        tags (List[str], optional): List of tags to associate with the tool.

    Returns:
        function: The wrapped function.
    """
    def decorator(func):
        # Use the reusable function to extract metadata
        metadata = get_tool_metadata(
            func=func,
            tool_name=tool_name,
            description=description,
            parameters_override=parameters_override,
            terminal=terminal,
            tags=tags
        )

        # Register the tool in the global dictionary
        tools[metadata["tool_name"]] = {
            "description": metadata["description"],
            "parameters": metadata["parameters"],
            "function": metadata["function"],
            "terminal": metadata["terminal"],
            "tags": metadata["tags"] or []
        }

        for tag in metadata["tags"]:
            if tag not in tools_by_tag:
                tools_by_tag[tag] = []
            tools_by_tag[tag].append(metadata["tool_name"])

        return func
    return decorator


@dataclass
class Prompt:
    messages: List[Dict] = field(default_factory=list)
    tools: List[Dict] = field(default_factory=list)
    metadata: dict = field(default_factory=dict)  # Fixing mutable default issue

def generate_response(prompt: Prompt) -> str:
    messages = prompt.messages
    tools = prompt.tools

    model = "groq/meta-llama/llama-4-scout-17b-16e-instruct"
    base_url = "https://api.groq.com/openai/v1"
    api_key = "API KEY"

    if not tools:
        response = completion(
            model=model,
            base_url=base_url,
            api_key=api_key,
            messages=messages,
            max_tokens=1024,
            temperature=0.0
        )
    else:
        response = completion(
            model=model,
            base_url=base_url,
            api_key=api_key,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            #function_call="auto",
            max_tokens=1500,
            temperature=0.0,
        )

    print("=== Full API Response ===")
    print(response)
    print("=========================")

    raw_output = response.choices[0].message.content

    print(f"Raw model output:\n{raw_output}\n--- End of output ---\n")

    # Fallback if model output is None or empty string
    if not raw_output or raw_output.strip() == "":
        # Return fallback action as JSON string to prompt listing files again
        fallback = json.dumps({"tool": "list_project_files", "args": {}})
        print("[Warning] Model returned empty response. Using fallback:", fallback)
        return fallback

    return raw_output



# def generate_response(prompt: Prompt) -> str:
#     """Call LLM to get response"""

#     messages = prompt.messages
#     tools = prompt.tools

#     result = None

#     if not tools:
#         response = completion(
#             model="openai/gpt-4o",
#             messages=messages,
#             max_tokens=1024
#         )
#         result = response.choices[0].message.content
#     else:
#         response = completion(
#             model="openai/gpt-4o",
#             messages=messages,
#             tools=tools,
#             max_tokens=1024
#         )

#         if response.choices[0].message.tool_calls:
#             tool = response.choices[0].message.tool_calls[0]
#             result = {
#                 "tool": tool.function.name,
#                 "args": json.loads(tool.function.arguments),
#             }
#             result = json.dumps(result)
#         else:
#             result = response.choices[0].message.content


#     return result


@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str


class Action:
    def __init__(self,
                 name: str,
                 function: Callable,
                 description: str,
                 parameters: Dict,
                 terminal: bool = False):
        self.name = name
        self.function = function
        self.description = description
        self.terminal = terminal
        self.parameters = parameters

    def execute(self, **args) -> Any:
        """Execute the action's function"""
        return self.function(**args)


class ActionRegistry:
    def __init__(self):
        self.actions = {}

    def register(self, action: Action):
        self.actions[action.name] = action

    def get_action(self, name: str) -> Optional[Action]:
        return self.actions.get(name, None)

    # def get_action(self, response):
    #   invocation = self.agent_language.parse_response(response)

    #   if not isinstance(invocation, dict):
    #       raise ValueError(f"parse_response did not return a dict: got {invocation} (type: {type(invocation)})")
    #   if "tool" not in invocation:
    #       raise ValueError(f"'tool' key not found in invocation: {invocation}")

    #   action = self.actions.get_action(invocation["tool"])
    #   return action, invocation


    def get_actions(self) -> List[Action]:
        """Get all registered actions"""
        return list(self.actions.values())


class Memory:
    def __init__(self):
        self.items = []  # Basic conversation histor

    def add_memory(self, memory: dict):
        """Add memory to working memory"""
        self.items.append(memory)

    def get_memories(self, limit: int = None) -> List[Dict]:
        """Get formatted conversation history for prompt"""
        return self.items[:limit]

    def copy_without_system_memories(self):
        """Return a copy of the memory without system memories"""
        filtered_items = [m for m in self.items if m["type"] != "system"]
        memory = Memory()
        memory.items = filtered_items
        return memory


class Environment:
    def execute_action(self, action: Action, args: dict) -> dict:
        """Execute an action and return the result."""
        try:
            result = action.execute(**args)
            return self.format_result(result)
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e),
                "traceback": traceback.format_exc()
            }

    def format_result(self, result: Any) -> dict:
        """Format the result with metadata."""
        return {
            "tool_executed": True,
            "result": result,
            "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")
        }


class AgentLanguage:
    def __init__(self):
        pass

    def construct_prompt(self,
                         actions: List[Action],
                         environment: Environment,
                         goals: List[Goal],
                         memory: Memory) -> Prompt:
        raise NotImplementedError("Subclasses must implement this method")


    def parse_response(self, response: str) -> dict:
        raise NotImplementedError("Subclasses must implement this method")



class AgentFunctionCallingActionLanguage(AgentLanguage):

    def __init__(self):
        super().__init__()

    # def format_goals(self, goals: List[Goal]) -> List:
    #     # Map all goals to a single string that concatenates their description
    #     # and combine into a single message of type system
    #     sep = "\n-------------------\n"
    #     goal_instructions = "\n\n".join([f"{goal.name}:{sep}{goal.description}{sep}" for goal in goals])
    #     return [
    #         {"role": "system", "content": goal_instructions}
    #     ]

    # def format_memory(self, memory: Memory) -> List:
    #     """Generate response from language model"""
    #     # Map all environment results to a role:user messages
    #     # Map all assistant messages to a role:assistant messages
    #     # Map all user messages to a role:user messages
    #     items = memory.get_memories()
    #     mapped_items = []
    #     for item in items:

    #         content = item.get("content", None)
    #         if not content:
    #             content = json.dumps(item, indent=4)

    #         if item["type"] == "assistant":
    #             mapped_items.append({"role": "assistant", "content": content})
    #         elif item["type"] == "environment":
    #             mapped_items.append({"role": "assistant", "content": content})
    #         else:
    #             mapped_items.append({"role": "user", "content": content})

    #     return mapped_items



    def format_goals(self, goals: List[Goal]) -> List:
          sep = "\n-------------------\n"
          goal_instructions = "\n\n".join([f"{goal.name}:{sep}{goal.description}{sep}" for goal in goals])

          # Few-shot instructions + example tool calling sequence
          few_shot_instructions = """
      You have the following tools available:

      1. list_project_files() - no parameters, returns a list of Python files.
      2. read_project_file(name: str) - reads the content of a specified file.
      3. terminate(message: str) - writes the README file and ends the session.

      Your task is to:

      - First call list_project_files with an empty argument object: {"tool": "list_project_files", "args": {}}
      - Then for each file in the result, call read_project_file with its filename, e.g.: {"tool": "read_project_file", "args": {"name": "example.py"}}
      - After reading all files, call terminate with the README content, e.g.: {"tool": "terminate", "args": {"message": "README content here"}}

      Respond **only** with a JSON-formatted function call as shown above. Do not write explanations or any other text.

      Below is an example sequence of calls:

      1. {"tool": "list_project_files", "args": {}}
      2. {"tool": "read_project_file", "args": {"name": "cl3-7.py"}}
      3. {"tool": "read_project_file", "args": {"name": "cl3-clone.py"}}
      4. {"tool": "read_project_file", "args": {"name": "deap_local.py"}}
      5. {"tool": "terminate", "args": {"message": "Comprehensive README content summarizing the project."}}

      Proceed step-by-step and do not skip any steps.
      """

          full_instruction = goal_instructions + "\n\n" + few_shot_instructions

          return [
              {"role": "system", "content": full_instruction}
          ]


    def format_actions(self, actions: List[Action]) -> [List,List]:
        """Generate response from language model"""

        tools = [
            {
                "type": "function",
                "function": {
                    "name": action.name,
                    # Include up to 1024 characters of the description
                    "description": action.description[:1024],
                    "parameters": action.parameters,
                },
            } for action in actions
        ]

        return tools

    def format_memory(self, memory: Memory) -> List:
        items = memory.get_memories()
        mapped_items = []
        for item in items:
            content = item.get("content")
            if not content:
                content = json.dumps(item, indent=4)

            if item["type"] == "environment":
                try:
                    parsed = json.loads(content)
                    pretty_content = json.dumps(parsed, indent=2)
                    content = f"Environment response:\n{pretty_content}"
                except Exception:
                    pass
                mapped_items.append({"role": "assistant", "content": content})
            elif item["type"] == "assistant":
                mapped_items.append({"role": "assistant", "content": content})
            else:
                mapped_items.append({"role": "user", "content": content})
        return mapped_items


    def construct_prompt(self,
                         actions: List[Action],
                         environment: Environment,
                         goals: List[Goal],
                         memory: Memory) -> Prompt:

        prompt = []
        prompt += self.format_goals(goals)
        prompt += self.format_memory(memory)

        tools = self.format_actions(actions)

        return Prompt(messages=prompt, tools=tools)

    def adapt_prompt_after_parsing_error(self,
                                         prompt: Prompt,
                                         response: str,
                                         traceback: str,
                                         error: Any,
                                         retries_left: int) -> Prompt:

        return prompt

    # def parse_response(self, response: str) -> dict:
    #     """Parse LLM response into structured format by extracting the ```json block"""

    #     try:
    #         return json.loads(response)

    #     except Exception as e:
    #         return {
    #             "tool": "terminate",
    #             "args": {"message":response}
    #         }

    def parse_response(self, response: str) -> dict:
        try:
            data = json.loads(response)
            if not isinstance(data, dict) or isinstance(data, list):
                print(f"[Warning] Unexpected response format: {data}")
                return {"tool": "list_project_files", "args": {}}
            return data
        except Exception as e:
            print(f"[Error] JSON parse failed: {e}")
            return {"tool": "list_project_files", "args": {}}



class PythonActionRegistry(ActionRegistry):
    def __init__(self, tags: List[str] = None, tool_names: List[str] = None):
        super().__init__()

        self.terminate_tool = None

        for tool_name, tool_desc in tools.items():
            if tool_name == "terminate":
                self.terminate_tool = tool_desc

            if tool_names and tool_name not in tool_names:
                continue

            tool_tags = tool_desc.get("tags", [])
            if tags and not any(tag in tool_tags for tag in tags):
                continue

            self.register(Action(
                name=tool_name,
                function=tool_desc["function"],
                description=tool_desc["description"],
                parameters=tool_desc.get("parameters", {}),
                terminal=tool_desc.get("terminal", False)
            ))

    def register_terminate_tool(self):
        if self.terminate_tool:
            self.register(Action(
                name="terminate",
                function=self.terminate_tool["function"],
                description=self.terminate_tool["description"],
                parameters=self.terminate_tool.get("parameters", {}),
                terminal=self.terminate_tool.get("terminal", False)
            ))
        else:
            raise Exception("Terminate tool not found in tool registry")



class Agent:
    def __init__(self,
                 goals: List[Goal],
                 agent_language: AgentLanguage,
                 action_registry: ActionRegistry,
                 generate_response: Callable[[Prompt], str],
                 environment: Environment):
        """
        Initialize an agent with its core GAME components
        """
        self.goals = goals
        self.generate_response = generate_response
        self.agent_language = agent_language
        self.actions = action_registry
        self.environment = environment

    def construct_prompt(self, goals: List[Goal], memory: Memory, actions: ActionRegistry) -> Prompt:
        """Build prompt with memory context"""
        return self.agent_language.construct_prompt(
            actions=actions.get_actions(),
            environment=self.environment,
            goals=goals,
            memory=memory
        )

    # def get_action(self, response):
    #     invocation = self.agent_language.parse_response(response)
    #     action = self.actions.get_action(invocation["tool"])
    #     return action, invocation

    def get_action(self, response: str):
        invocation = self.agent_language.parse_response(response)

        if not isinstance(invocation, dict):
            raise ValueError(f"parse_response did not return a dict: got {invocation} (type: {type(invocation)})")
        if "tool" not in invocation:
            raise ValueError(f"'tool' key not found in invocation: {invocation}")

        action = self.actions.get_action(invocation["tool"])
        return action, invocation

    def should_terminate(self, response: str) -> bool:
        action_def, _ = self.get_action(response)
        return action_def.terminal

    def set_current_task(self, memory: Memory, task: str):
        memory.add_memory({"type": "user", "content": task})

    def update_memory(self, memory: Memory, response: str, result: dict):
        """
        Update memory with the agent's decision and the environment's response.
        """
        new_memories = [
            {"type": "assistant", "content": response},
            {"type": "environment", "content": json.dumps(result)}
        ]
        for m in new_memories:
            memory.add_memory(m)

    def prompt_llm_for_action(self, full_prompt: Prompt) -> str:
        response = self.generate_response(full_prompt)
        return response

    def run(self, user_input: str, memory=None, max_iterations: int = 50) -> Memory:
        """
        Execute the GAME loop for this agent with a maximum iteration limit.
        """
        memory = memory or Memory()
        self.set_current_task(memory, user_input)

        for _ in range(max_iterations):
            # Construct a prompt that includes the Goals, Actions, and the current Memory
            prompt = self.construct_prompt(self.goals, memory, self.actions)

            print("Agent thinking...")
            # Generate a response from the agent
            response = self.prompt_llm_for_action(prompt)
            print(f"Agent Decision: {response}")

            # Determine which action the agent wants to execute
            action, invocation = self.get_action(response)

            # Execute the action in the environment
            result = self.environment.execute_action(action, invocation["args"])
            print(f"Action Result: {result}")

            # Update the agent's memory with information about what happened
            self.update_memory(memory, response, result)

            # Check if the agent has decided to terminate
            if self.should_terminate(response):
                break

        return memory


In [35]:

    from typing import Any, Dict, List, Optional
    import os

    class ActionContext:
        """
        Simple context object passed to tools/functions to provide runtime metadata and state.

        Attributes:
            current_user_input (Optional[str]): Latest user input or task description.
            memory (List[Dict[str, Any]]): List of past conversation or environment interactions.
            goals (List[Any]): Current goals of the agent.
            environment_vars (Dict[str, Any]): Environment or configuration variables.
            user_role (Optional[str]): Current user's role or permissions.
        """

        def __init__(
            self,
            current_user_input: Optional[str] = None,
            memory: Optional[List[Dict[str, Any]]] = None,
            goals: Optional[List[Any]] = None,
            environment_vars: Optional[Dict[str, Any]] = None,
            user_role: Optional[str] = None,
        ):
            self.current_user_input = current_user_input
            self.memory = memory if memory is not None else []
            self.goals = goals if goals is not None else []
            self.environment_vars = environment_vars if environment_vars is not None else {}
            self.user_role = user_role

        def add_to_memory(self, item: Dict[str, Any]) -> None:
            """Add a memory item to the context's memory list."""
            self.memory.append(item)

        def log(self, message: str) -> None:
            """Simple log method; in real use replace with proper logging."""
            print(f"[ActionContext LOG]: {message}")


    # First, we'll define our tools using decorators
    @register_tool(tags=["file_operations", "list"])
    def list_project_files() -> List[str]:
        """Lists all Python files in the current project directory.

        Scans the current directory and returns a sorted list of all files
        that end with '.py'.

        Returns:
            A sorted list of Python filenames
        """
        return sorted([file for file in os.listdir(".")
                      if file.endswith(".py")])

    #@register_tool(tags=["file_operations", "read"])
    @register_tool(
    tags=["file_operations", "read"],
    description="Read the content of a project file by filename.",
    parameters_override={
        "type": "object",
        "properties": {
            "name": {"type": "string", "description": "The filename to read, including extension."}
        },
        "required": ["name"]
    }
)
    def read_project_file(name: str) -> str:
            """Reads and returns the content of a specified project file.

            Opens the file in read mode and returns its entire contents as a string.
            Raises FileNotFoundError if the file doesn't exist.

            Args:
                name: The name of the file to read

            Returns:
                The contents of the file as a string
            """
            with open(name, "r") as f:
                return f.read()



    # @register_tool(tags=["documentation"])
    # def generate_technical_documentation(action_context: ActionContext, code_or_feature: str) -> str:
    #     """Generate technical documentation by consulting a senior technical writer."""
    #     return prompt_expert(
    #         action_context=action_context,
    #         description_of_expert="""
    #         You are a senior technical writer with 15 years of experience in software documentation...
    #         """,
    #         prompt=f"""
    #         Please create comprehensive technical documentation for the following code or feature:

    #         {code_or_feature}

    #         Your documentation should include:
    #         1. A clear overview...
    #         2. Detailed explanation...
    #         3. Key interfaces...
    #         ...
    #         """
    #     )

    # @register_tool(tags=["testing"])
    # def design_test_suite(action_context: ActionContext, feature_description: str) -> str:
    #     """Design a comprehensive test suite by consulting a senior QA engineer."""
    #     return prompt_expert(
    #         action_context=action_context,
    #         description_of_expert="""
    #         You are a senior QA engineer with 12 years of experience...
    #         """,
    #         prompt=f"""
    #         Please design a comprehensive test suite for the following feature:

    #         {feature_description}

    #         Your test design should cover:
    #         1. Unit tests...
    #         2. Integration tests...
    #         ...
    #         """
    #     )

    # @register_tool(tags=["code_quality"])
    # def perform_code_review(action_context: ActionContext, code: str) -> str:
    #     """Review code and suggest improvements by consulting a senior software architect."""
    #     return prompt_expert(
    #         action_context=action_context,
    #         description_of_expert="""
    #         You are a senior software architect with 20 years of experience...
    #         """,
    #         prompt=f"""
    #         Please review the following code and provide detailed improvement suggestions:

    #         {code}

    #         Consider and address:
    #         1. Code organization...
    #         ...
    #         """
    #     )

    # @register_tool(tags=["communication"])
    # def write_feature_announcement(action_context: ActionContext, feature_details: str, audience: str) -> str:
    #     """Write a feature announcement by consulting a product marketing expert."""
    #     return prompt_expert(
    #         action_context=action_context,
    #         description_of_expert="""
    #         You are a senior product marketing manager with 12 years of experience...
    #         """,
    #         prompt=f"""
    #         Please write a feature announcement for the following feature:

    #         {feature_details}

    #         This announcement is intended for a {audience} audience.

    #         Your announcement should include:
    #         1. A compelling introduction...
    #         ...
    #         """
    #     )

    # @register_tool(tags=["system"], terminal=True)
    # def terminate(readme_content: str) -> str:
    #     """Writes a README.md file and terminates the agent."""
    #     with open("README.md", "w") as f:
    #         f.write(readme_content)
    #     return f"README.md successfully written with content:\n\n{readme_content}"



    @register_tool(tags=["documentation"])
    def generate_technical_documentation(action_context: ActionContext, code_or_feature: str) -> str:
        """
        Generate technical documentation by consulting a senior technical writer.
        This expert focuses on creating clear, comprehensive documentation for developers.

        Args:
            code_or_feature: The code or feature to document
        """
        return prompt_expert(
            action_context=action_context,
            description_of_expert="""
            You are a senior technical writer with 15 years of experience in software documentation.
            You have particular expertise in:
            - Writing clear and precise API documentation
            - Explaining complex technical concepts to developers
            - Documenting implementation details and integration points
            - Creating code examples that illustrate key concepts
            - Identifying and documenting important caveats and edge cases

            Your documentation is known for striking the perfect balance between completeness
            and clarity. You understand that good technical documentation serves as both
            a reference and a learning tool.
            """,
            prompt=f"""
            Please create comprehensive technical documentation for the following code or feature:

            {code_or_feature}

            Your documentation should include:
            1. A clear overview of the feature's purpose and functionality
            2. Detailed explanation of the implementation approach
            3. Key interfaces and integration points
            4. Usage examples with code snippets
            5. Important considerations and edge cases
            6. Performance implications if relevant

            Focus on providing information that developers need to effectively understand
            and work with this code.
            """
        )

    @register_tool(tags=["testing"])
    def design_test_suite(action_context: ActionContext, feature_description: str) -> str:
        """
        Design a comprehensive test suite by consulting a senior QA engineer.
        This expert focuses on creating thorough test coverage with attention to edge cases.

        Args:
            feature_description: Description of the feature to test
        """
        return prompt_expert(
            action_context=action_context,
            description_of_expert="""
            You are a senior QA engineer with 12 years of experience in test design and automation.
            Your expertise includes:
            - Comprehensive test strategy development
            - Unit, integration, and end-to-end testing
            - Performance and stress testing
            - Security testing considerations
            - Test automation best practices

            You are particularly skilled at identifying edge cases and potential failure modes
            that others might miss. Your test suites are known for their thoroughness and
            their ability to catch issues early in the development cycle.
            """,
            prompt=f"""
            Please design a comprehensive test suite for the following feature:

            {feature_description}

            Your test design should cover:
            1. Unit tests for individual components
            2. Integration tests for component interactions
            3. End-to-end tests for critical user paths
            4. Performance test scenarios if relevant
            5. Edge cases and error conditions
            6. Test data requirements

            For each test category, provide:
            - Specific test scenarios
            - Expected outcomes
            - Important edge cases to consider
            - Potential testing challenges
            """
        )

    @register_tool(tags=["code_quality"])
    def perform_code_review(action_context: ActionContext, code: str) -> str:
        """
        Review code and suggest improvements by consulting a senior software architect.
        This expert focuses on code quality, architecture, and best practices.

        Args:
            code: The code to review
        """
        return prompt_expert(
            action_context=action_context,
            description_of_expert="""
            You are a senior software architect with 20 years of experience in code review
            and software design. Your expertise includes:
            - Software architecture and design patterns
            - Code quality and maintainability
            - Performance optimization
            - Scalability considerations
            - Security best practices

            You have a talent for identifying subtle design issues and suggesting practical
            improvements that enhance code quality without over-engineering.
            """,
            prompt=f"""
            Please review the following code and provide detailed improvement suggestions:

            {code}

            Consider and address:
            1. Code organization and structure
            2. Potential design pattern applications
            3. Performance optimization opportunities
            4. Error handling completeness
            5. Edge case handling
            6. Maintainability concerns

            For each suggestion:
            - Explain the current issue
            - Provide the rationale for change
            - Suggest specific improvements
            - Note any trade-offs to consider
            """
        )

    @register_tool(tags=["communication"])
    def write_feature_announcement(action_context: ActionContext,
                                feature_details: str,
                                audience: str) -> str:
        """
        Write a feature announcement by consulting a product marketing expert.
        This expert focuses on clear communication of technical features to different audiences.

        Args:
            feature_details: Technical details of the feature
            audience: Target audience for the announcement (e.g., "technical", "business")
        """
        return prompt_expert(
            action_context=action_context,
            description_of_expert="""
            You are a senior product marketing manager with 12 years of experience in
            technical product communication. Your expertise includes:
            - Translating technical features into clear value propositions
            - Crafting compelling product narratives
            - Adapting messaging for different audience types
            - Building excitement while maintaining accuracy
            - Creating clear calls to action

            You excel at finding the perfect balance between technical accuracy and
            accessibility, ensuring your communications are both precise and engaging.
            """,
            prompt=f"""
            Please write a feature announcement for the following feature:

            {feature_details}

            This announcement is intended for a {audience} audience.

            Your announcement should include:
            1. A compelling introduction
            2. Clear explanation of the feature
            3. Key benefits and use cases
            4. Technical details (adapted to audience)
            5. Implementation requirements
            6. Next steps or call to action

            Ensure the tone and technical depth are appropriate for a {audience} audience.
            Focus on conveying both the value and the practical implications of this feature.
            """
        )

    # @register_tool(tags=["system"], terminal=True)
    # def terminate(readme_content: str) -> str:
    #         """Writes a README.md file and terminates the agent."""
    #         with open("README (4).md", "w") as f:
    #             f.write(readme_content)
    #         return f"README.md successfully written with content:\n\n{readme_content}"

    @register_tool(tags=["system"], terminal=True)
    def terminate(message: str) -> str:
        with open("README.md", "w") as f:
            f.write(message)
        return f"README.md successfully written with content:\n\n{message}"



"""
    @register_tool(tags=["system"], terminal=True)
    def terminate(message: str) -> str:
        # Terminates the agent's execution with a final message.

        # Args:
        #     message: The final message to return before terminating

        # Returns:
        #     The message with a termination note appended
        """
        # return f"{message}\nTerminating..."


# Define the agent's goals
goals = [
    Goal(
        priority=1,
        name="List Project Files",
        description=(
            "Use the 'list_project_files' tool to get a list of all Python files in the project directory."
            " This will allow you to systematically process each file."
        )
    ),
    Goal(
        priority=1,
        name="Read Project Files",
        description=(
            "For each file obtained from the file listing, use the 'read_project_file' tool to read its contents."
            " Store and integrate this information as part of your knowledge about the project."
        )
    ),
    Goal(
        priority=1,
        name="Generate README",
        description=(
            "After reading all files, generate comprehensive technical documentation summarizing the"
            " project’s functionality, key features, and usage."
        )
    ),
    Goal(
        priority=1,
        name="Terminate",
        description=(
            "Call the 'terminate' tool when you have prepared the complete README content."
        )
    ),
]



    # Create an agent instance with tag-filtered actions
agent = Agent(
        goals=goals,
        agent_language=AgentFunctionCallingActionLanguage(),
        # The ActionRegistry now automatically loads tools with these tags
        action_registry=PythonActionRegistry(tags=["file_operations", "read","system"]),
        generate_response=generate_response,
        environment=Environment()
    )

    # Run the agent with user input
user_input = "Your process: 1. Use list_project_files. 2. For each file, use read_project_file. 3. After all files, call terminate with a README based on the contents you've read."
final_memory = agent.run(user_input, max_iterations=100)
print(final_memory.get_memories())

Agent thinking...
=== Full API Response ===
ModelResponse(id='chatcmpl-2e77eeff-0956-4fe9-b7d3-34faea60543b', created=1753476889, model='meta-llama/llama-4-scout-17b-16e-instruct', object='chat.completion', system_fingerprint='fp_37da608fc1', choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{}', name='list_project_files'), id='p257zdz2y', type='function')], function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=17, prompt_tokens=1263, total_tokens=1280, completion_tokens_details=None, prompt_tokens_details=None, queue_time=0.212414775, prompt_time=0.037416927, completion_time=0.043729362, total_time=0.081146289), usage_breakdown=None, x_groq={'id': 'req_01k11nvz3mekyr4b7mq2qy3xpk'}, service_tier='auto')
Raw model output:
None
--- End of output ---

[Warning] Model returned empty response. Using fallback: {"tool": "list_project_files", 

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `meta-llama/llama-4-scout-17b-16e-instruct` in organization `org_01jydvpy3be018597m66jqf198` service tier `on_demand` on tokens per minute (TPM): Limit 30000, Used 29258, Requested 1617. Please try again in 1.75s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}
